<a href="https://colab.research.google.com/github/Eng-DanielaNunes/collective-intelligence-lab/blob/main/notebooks/daninunes_decide%2B_2025_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Projeto: DECIDE + - Plataforma de Apoio à Decisão para Alocação Estratégica de Projetos Tecnológicos
# Autora: Daniela Nunes dos Santos Ferreira
# Ano de Desenvolvimento: 2025
# Versão: 1.0
# Descrição: Plataforma de Apoio à Decisão para Alocação Estratégica de Projetos Tecnológicos
# Objetivo: avaliar e ranquear os ISTs disponíveis de acordo com seu
# desempenho em diversos critérios, e então indicar o mais adequado para receber
# uma demanda de projetos.


# ===================== AVISO DE PROTEÇÃO =====================
# Este sistema está em fase BETA e protegido contra alterações.
# Qualquer modificação indevida pode comprometer os resultados.
# Em caso de dúvidas, contatar:
# Daniela Nunes –  Pesquisadora Mestre e Consultora de Negócios
# =============================================================


# ===== Passo - a passo do Modelo ========

#1. Define os ISTs disponíveis: ISTA, ISTMPP, ISI-TICs.
#2. Cria uma matriz de decisão, onde cada IST recebe uma nota de 1 a 5 em vários
# critérios usando a escala de Likert, como:


# ===== Critérios de Decisão =====
#•	Aderência Técnica.
#•	Disponibilidade e Capacidade.
#•	Prioridade Estratégica.
#•	Tempo Estimado de Execução (quanto menor, melhor).
#•	Capilaridade Territorial.
#•	Potencial de Inovação e Impacto.

# Os critérios podem variar conforme o cenário dos projetos.
# A análise deve considerar a tipicidade de cada contexto,
# podendo incluir novos critérios ou ajustar os existentes
# de acordo com a necessidade de maximização ou minimização.

# 3. Classificação.
# os critérios como de benefício (quanto maior, melhor) ou de custo (quanto
# menor, melhor).

# 4. Normaliza os dados, para deixar todos os critérios
# comparáveis (padroniza a escala).

# 5. Calcula os pesos automaticamente com base
# na importância relativa dos critérios, usando o método aditivo (os
# critérios com maior variação ou maior contribuição ganham mais peso).

# 6. Aplica o método TOPSIS:
# Calcula qual ISt está mais próximo da solução
# ideal (melhor em todos os critérios). E qual está mais distante da
# solução anti-ideal (pior em todos os critérios).
# Gera um score final para cada IST.

# 7. Ordena os ISTs por esse score – o primeiro do ranking é o
# mais aderente para receber a demanda.

# 8. Gera Estatísticas descritivas por critérios, gráficos e relatórios:
# Gráfico de barras com os scores TOPSIS. • Radar chart com os pontos
# fortes/fracos. • Heatmap dos critérios. • Exporta tudo para o Excel para
# compartilhamento e/ou tomada de decisão.


# 9. Execução do código completo: clique na primeira linha e, em seguida, aperte
# simultaneamente Ctrl F9 OU clique em Run all na barra de comandos
# OU Runtime e, em seguida, clique em Run all



# 10. Nota Técnica: Versão Beta do Sistema de Apoio à Decisão

# ==============================================================================
#
#                             DECIDE +
# ==============================================================================

# Instalação das bibliotecas (executar apenas se necessário)
!pip install pandas numpy matplotlib seaborn --quiet

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import pi

# === Função para entrada interativa ===
def entrada_interativa():
    print("=== Construção interativa da Matriz de Decisão ===\n")

    # Entrada dos ISTs
    while True:
        ists_input = input("Digite os nomes dos ISTs separados por vírgula e pressione a tecla ENTER (ex: ISTA, ISTMPP, ISI-TICs): ").strip()
        ists = [i.strip() for i in ists_input.split(",") if i.strip()]
        if len(ists) >= 2:
            break
        print("Por favor, insira pelo menos dois ISTs.\n")

    # Entrada dos critérios
    while True:
        criterios_input = input("Digite os critérios separados por vírgula e pressione a tecla ENTER  (ex: Aderência Técnica, Disponibilidade): ").strip()
        criterios = [c.strip() for c in criterios_input.split(",") if c.strip()]
        if len(criterios) >= 2:
            break
        print("Por favor, insira pelo menos dois critérios.\n")

    # Definir se cada critério é benefício ou custo
    benefit_criteria = {}
    print("\nPara cada critério, informe se é BENEFÍCIO (maximizar) ou CUSTO (minimizar) e pressione a tecla ENTER.")
    for c in criterios:
        while True:
            resp = input(f"Critério '{c}': (B)enefício ou (C)usto? ").strip().upper()
            if resp in ['B', 'C']:
                benefit_criteria[c] = (resp == 'B')
                break
            print("Resposta inválida. Digite 'B' para benefício ou 'C' para custo.")

    # Coletar notas para cada IST e critério
    print("\nInforme as notas para cada IST e critério (valores numéricos, ex: 7, 8.5, 10) e e pressione a tecla ENTER .")
    dados = {}
    for ist in ists:
        notas_ist = []
        print(f"\nNotas para IST: {ist}")
        for c in criterios:
            while True:
                try:
                    val = float(input(f"  Nota para critério '{c}': "))
                    if val < 0:
                        print("Valor deve ser positivo.")
                        continue
                    notas_ist.append(val)
                    break
                except:
                    print("Entrada inválida. Digite um número.")
        dados[ist] = notas_ist

    # Montar DataFrame
    decision_matrix = pd.DataFrame(dados, index=criterios).T

    return decision_matrix, benefit_criteria

# === Executar entrada interativa ===
decision_matrix, benefit_criteria = entrada_interativa()

print("\nMatriz de Decisão:")
display(decision_matrix)

print("\nCritérios (Benefício = True, Custo = False):")
print(benefit_criteria)


# Estatísticas Descritiivas básicas por Critério

# Estatísticas Descritivas Básicas por Critério
desc_stats = decision_matrix.describe().T
desc_stats['median'] = decision_matrix.median()
desc_stats['variance'] = decision_matrix.var()
desc_stats['coef_var_%'] = (decision_matrix.std() / decision_matrix.mean()) * 100

print("\n=== Estatísticas Descritivas por Critério ===")
print(desc_stats[['mean', 'median', 'std', 'variance', 'coef_var_%']].round(3))

# Matriz de Correlação entre Critérios
corr_matrix = decision_matrix.corr()
print("\n=== Matriz de Correlação entre Critérios ===")
print(corr_matrix.round(3))

# Visualização da Matriz de Correlação com Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt=".2f")
plt.title("Matriz de Correlação entre Critérios")
plt.show()

# Boxplots por Critério para visualizar dispersão e outliers
plt.figure(figsize=(10, 6))
sns.boxplot(data=decision_matrix)
plt.title("Boxplot dos Critérios dos ISTs")
plt.xticks(rotation=45)
plt.show()


# --- Resto do código --- (a partir da normalização, pesos, TOPSIS, gráficos...)

# === 4. Normalização da matriz de decisão ===
norm_matrix = decision_matrix.copy()
for column in decision_matrix.columns:
    if benefit_criteria[column]:
        norm_matrix[column] = decision_matrix[column] / np.sqrt((decision_matrix[column]**2).sum())
    else:
        norm_matrix[column] = (1 / decision_matrix[column]) / np.sqrt(((1 / decision_matrix[column])**2).sum())

# === 5. Definição dos pesos dos critérios usando o método aditivo ===
column_sums = pd.Series(dtype=float)
for column in norm_matrix.columns:
    if benefit_criteria[column]:
        column_sums[column] = norm_matrix[column].sum()
    else:
        column_sums[column] = 1 / norm_matrix[column].sum()

weights = column_sums / column_sums.sum()

print("\nPesos dos critérios (método aditivo):")
print(weights.round(4))

weighted_matrix = norm_matrix * weights

# === 6. Soluções ideal e anti-ideal ===
# Distãncia das soluções Ideal e Anti-Ideal: mostra o quanto cada IST está próximo da melhor (ideal) e da pior (anti-ideal) situação
ideal_solution = []
anti_ideal_solution = []
for col, is_benefit in benefit_criteria.items():
    if is_benefit:
        ideal_solution.append(weighted_matrix[col].max())
        anti_ideal_solution.append(weighted_matrix[col].min())
    else:
        ideal_solution.append(weighted_matrix[col].min())
        anti_ideal_solution.append(weighted_matrix[col].max())
ideal_solution = np.array(ideal_solution)
anti_ideal_solution = np.array(anti_ideal_solution)

# === 7. Distâncias à solução ideal e anti-ideal ===
dist_to_ideal = np.sqrt(((weighted_matrix - ideal_solution) ** 2).sum(axis=1))
dist_to_anti_ideal = np.sqrt(((weighted_matrix - anti_ideal_solution) ** 2).sum(axis=1))

# === 8. Cálculo do score TOPSIS ===
topsis_score = dist_to_anti_ideal / (dist_to_ideal + dist_to_anti_ideal)

# === 9. Exibir resultado final ordenado ===
topsis_result = pd.DataFrame({
    'IST': decision_matrix.index,
    'Topsis Score': topsis_score
}).sort_values(by='Topsis Score', ascending=False).reset_index(drop=True)

print("\nRanking dos ISTs - Score TOPSIS:")
display(topsis_result)

# === 10. Gráfico visual do ranking ===
colors = sns.color_palette("Set2", n_colors=len(topsis_result))
plt.figure(figsize=(8, 5))
plt.bar(topsis_result['IST'], topsis_result['Topsis Score'], color=colors)
plt.title('Ranking dos ISTs - Score TOPSIS')
plt.ylabel('Score TOPSIS')
plt.ylim(0, 1)
plt.grid(axis='y', linestyle='--', alpha=0.7)
for i, v in enumerate(topsis_result['Topsis Score']):
    plt.text(i, v + 0.02, f"{v:.2f}", ha='center', fontweight='bold')
plt.tight_layout()
plt.show()

# === 11. Interpretação do Ranking ===
def interpretar_ranking(ist_scores: dict):
    medalhas = ["🥇", "🥈", "🥉"]
    ranking = sorted(ist_scores.items(), key=lambda x: x[1], reverse=True)

    print("\n📊 Ranking Final dos ISTs\n")
    print(f"| {'Posição':<8} | {'IST':<15} | {'Score':<5} | Interpretação                                               |")
    print(f"|{'-'*10}|{'-'*17}|{'-'*7}|{'-'*60}|")

    for i, (ist, score) in enumerate(ranking):
        if i == 0:
            interp = "Mais próximo da solução ideal. Mais recomendado para liderar o projeto."
        elif i == 1:
            interp = f"Desempenho intermediário. Menos ideal que {ranking[0][0]}, mas melhor que {ranking[2][0]}."
        else:
            interp = "Mais distante da solução ideal. Menor score entre os ISTs."
        print(f"| {medalhas[i]} {i+1}º   | {ist:<15} | {score:<5.2f} | {interp:<60} |")

interpretar_ranking(dict(zip(topsis_result['IST'], topsis_result['Topsis Score'])))


# ===================================================================================================================
# ===== Comparação dos Pontos Fortes e Fracos de Cada ISI =====

# Radar Chart para os ISTs
# Compara os ISTs em todos os critérios numa escala padronizada de 0 a 10
categories = list(decision_matrix.columns)
N = len(categories)

# Normalizar os dados para o radar (0-10 escala)
normed_for_radar = decision_matrix.copy()
for col in normed_for_radar.columns:
    if benefit_criteria[col]:
        normed_for_radar[col] = (normed_for_radar[col] - normed_for_radar[col].min()) / (normed_for_radar[col].max() - normed_for_radar[col].min()) * 10
    else:
        normed_for_radar[col] = (normed_for_radar[col].max() - normed_for_radar[col]) / (normed_for_radar[col].max() - normed_for_radar[col].min()) * 10

# Plot radar chart
plt.figure(figsize=(8, 6))
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

for i in range(len(normed_for_radar)):
    values = normed_for_radar.iloc[i].tolist()
    values += values[:1]
    plt.polar(angles, values, label=normed_for_radar.index[i])

plt.xticks(angles[:-1], categories, fontsize=9)
plt.title('Comparação de Critérios por IST (Escala 0-10)')
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
plt.tight_layout()
plt.show()

# ===== Heatmap de Desempenho =====
# Exibe o desempenho bruto dos ISTs por critério

plt.figure(figsize=(10, 4))
sns.heatmap(decision_matrix, annot=True, cmap="YlGnBu", fmt=".1f")
plt.title("Desempenho dos ISTs por Critério")
plt.show()

# ==== Tabela de Contribuição por Critério (Score Parcial) ====

partial_scores = weighted_matrix.copy()
partial_scores['Total Score'] = topsis_score
display(partial_scores.round(4))

# Indicador de Gap para Solução Ideal

plt.figure(figsize=(8, 5))
plt.plot(topsis_result['IST'], dist_to_ideal[topsis_result['IST']], marker='o', color='red', label='Distância da Solução Ideal')
plt.plot(topsis_result['IST'], dist_to_anti_ideal[topsis_result['IST']], marker='o', color='green', label='Distância da Solução Anti-ideal')
plt.ylabel("Distância")
plt.title("Gap dos ISTs em relação às soluções ideal e anti-ideal")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# Exportação para Excel com Ranking Final

# === Exportação para Excel com todas as métricas avaliadas ===

with pd.ExcelWriter("Relatorio_TOPSIS_ISTs.xlsx") as writer:
    decision_matrix.to_excel(writer, sheet_name="01_Matriz de Decisão")
    norm_matrix.to_excel(writer, sheet_name="02_Matriz Normalizada")
    weighted_matrix.to_excel(writer, sheet_name="03_Matriz Ponderada")

    # Exportar os pesos dos critérios
    weights_df = weights.reset_index()
    weights_df.columns = ['Critério', 'Peso']
    weights_df.to_excel(writer, sheet_name="04_Pesos dos Critérios", index=False)

    # Exportar distâncias
    distancias_df = pd.DataFrame({
        'IST': decision_matrix.index,
        'Distância Ideal': dist_to_ideal,
        'Distância Anti-Ideal': dist_to_anti_ideal
    })
    distancias_df.to_excel(writer, sheet_name="05_Distâncias", index=False)

    # Exportar score final
    topsis_result.to_excel(writer, sheet_name="06_Ranking Final", index=False)

    # Exportar score parcial por critério (weighted matrix com score final)
    partial_scores.to_excel(writer, sheet_name="07_Score Parcial", index=True)


# BAIXAR A PLANILHA

from google.colab import files
files.download("Relatorio_TOPSIS_ISTs.xlsx")



=== Construção interativa da Matriz de Decisão ===

